Input and output available in 'SQLite3-Problem2.xlsx'

In [1]:
import pandas as pd
import sqlite3 as db

In [2]:
conn = db.connect('sqliteprac.db')

In [3]:
cn = conn.cursor()

In [24]:
cn.execute("create table entries ( \
name varchar(20),\
address varchar(20),\
email varchar(20),\
floor int,\
resources varchar(10));")

In [43]:
cn.execute("insert into entries values ('A','Bangalore','A@gmail.com',1,'CPU'),('A','Bangalore','A1@gmail.com',1,'CPU'),('A','Bangalore','A2@gmail.com',2,'DESKTOP')\
,('B','Bangalore','B@gmail.com',2,'DESKTOP'),('B','Bangalore','B1@gmail.com',2,'DESKTOP'),('B','Bangalore','B2@gmail.com',1,'MONITOR')")

In [26]:
cn.execute("select * from entries")
print(cn.fetchall())

[('A', 'Bangalore', 'A@gmail.com', 1, 'CPU'), ('A', 'Bangalore', 'A1@gmail.com', 1, 'CPU'), ('A', 'Bangalore', 'A2@gmail.com', 2, 'DESKTOP'), ('B', 'Bangalore', 'B@gmail.com', 2, 'DESKTOP'), ('B', 'Bangalore', 'B1@gmail.com', 2, 'DESKTOP'), ('B', 'Bangalore', 'B2@gmail.com', 1, 'MONITOR')]


In [27]:
query = 'select * from entries'
pd.read_sql_query(query,conn)

,name,address,email,floor,resources
0,A,Bangalore,A@gmail.com,1,CPU
1,A,Bangalore,A1@gmail.com,1,CPU
2,A,Bangalore,A2@gmail.com,2,DESKTOP
3,B,Bangalore,B@gmail.com,2,DESKTOP
4,B,Bangalore,B1@gmail.com,2,DESKTOP
5,B,Bangalore,B2@gmail.com,1,MONITOR


In [34]:
#Query to count the number of visits by A and B in each floor
query = """
select name, 
       floor,
       count(1) as no_of_floor_visit
from entries 
group by name,floor """
pd.read_sql_query(query,conn)

,name,floor,no_of_floor_visit
0,A,1,2
1,A,2,1
2,B,1,1
3,B,2,2


In [44]:
#Query to rank the visits by A and B using rank() 
#partition by - for each name we find the rank, order by count(1) desc - rank 1 for highest number of count(1)
query = """
select name, 
       floor,
       count(1) as no_of_floor_visit,
       rank() over (partition by name order by count(1) desc) as rank
from entries 
group by name,floor """
pd.read_sql_query(query,conn)

,name,floor,no_of_floor_visit,rank
0,A,1,2,1
1,A,2,1,2
2,B,2,2,1
3,B,1,1,2


In [39]:
#Query to concat the distinct values in resources followed by group by name to determine the resource used by each person 
query = """
select name, 
       count(1) as total_visits, 
       group_concat(distinct(resources)) as resources_used 
from entries 
group by name """
pd.read_sql_query(query,conn)

,name,total_visits,resources_used
0,A,3,"CPU,DESKTOP"
1,B,3,"DESKTOP,MONITOR"


In [30]:
#Query to join the temp tables and filter it based on rank 1 as per the floor visit
query = """
with total_visits as
(
select name, 
       count(1) as total_visits, 
       group_concat(distinct(resources)) as resources_used 
from entries 
group by name
),
floor_visit as
(
select name, 
       floor,
       count(1) as no_of_floor_visit,
       rank() over (partition by name order by count(1) desc) as rank
from entries 
group by name,floor
)

select fv.name, 
       fv.floor as most_visited_floor,
       tv.total_visits,
       tv.resources_used  
from floor_visit fv
inner join total_visits tv on fv.name = tv.name 
where rank = 1
"""

pd.read_sql_query(query,conn)

,name,most_visited_floor,total_visits,resources_used
0,A,1,3,"CPU,DESKTOP"
1,B,2,3,"DESKTOP,MONITOR"


In [45]:
conn.commit()
conn.close()